<table style="width: 100%">
    <tr style="background: #ffffff">
        <td style="padding-top:25px;width: 180px"><img src="https://mci.edu/templates/mci/images/logo.svg" alt="Logo"></td>
        <td style="width: 100%">
            <div style="text-align:right; width: 100%; text-align:right"><font style="font-size:38px"><b>Grundlagen Programmierung</b></font></div>
            <div style="padding-top:0px; width: 100%; text-align:right"><font size="4"><b>WS 2022</b></font></div>
        </td>
    </tr>
</table>

---

# 8 Challenge: Reconstruction in Shotgun-Sequencing

![](https://en.wikipedia.org/wiki/De_novo_sequence_assemblers)

## 2.8 Challenge: Reconstruction in Shotgun-Sequencing

So far, we assumed to have full length DNA-Sequences available from a database or any other source. In fact, we do not have the technology yet to read long strands of DNA a direct storing them in a database. Instead, the DNA strand is broken down in shorter sequences for reading the results. However, we have to bring the short sequences in the right order again.

"Shotgun sequencing is a laboratory technique for determining the DNA sequence of an organism’s genome. The method involves randomly breaking up the genome into small DNA fragments that are sequenced individually. A computer program looks for overlaps in the DNA sequences, using them to reassemble the fragments in their correct order to reconstitute the genome." [genome.gov](https://www.genome.gov/genetics-glossary/Shotgun-Sequencing)


![](https://www.genome.gov/sites/default/files/media/images/tg/Shotgun-sequencing.jpg)


You have a list of fragmented DNA-snippets that have to be assembled in the right order. Your task is to implement an algorihm, that brings the DNA-sequence into the right order.





### 2.8.1 Grading

This is the basis for the final grade. 

- Passing the unit tests for the following functions:
  - ```CompareBeginningsTestCase``` - **20 %**
  - ```CompareEndingsTestCase```- **20%**
  - ```GlueFragmentsTestCase``` - **20 %**
- Implement the following function without unit test:
  - ```assemble_sequence()``` - **10 %**
- Coding style and comments - **20 %**
- Length of final sequence and speed of execution of ```assemble_sequence()```- **10 %**


### 2.8.2 Preparation of the Data Set

Next, we prepare the data set for the challenge. The following code splits the Virus-DNA (```full_dna_sequence```), we have seen before, into 500 shorter segements with length 200.

This results in a high average coverage of $\text{coverage} = N \times L / G  = 500 \times 200 / 2316 = 43,3$. So on average every base is read 43 times.

The average coverage for a whole genome can be calculated from the length of the original genome ($G$), the number of reads ($N$), and the average read length ($L$) as $N \times L / G$

The segments are storend in a ```list_of_fragments```. 

In [18]:
# fragment the full dna sequence

# A python package to generate random numbers
import random

# The length and the number of segments
fragment_length = 200
number_of_fragments = 500


full_dna_sequence = "CAAACCATTTGAATGGATGTCAATCCGACTCTACTTTTCCTAAAAATTCCAGCACAAAATGCCATAAGCACCACATTCCCTTATACTGGAGATCCTCCATACAGCCATGGAACAGGAACAGGATACACCATGGACACAGTAAACAGAACACACCAATACTCAGAAAAGGGGAAGTGGACAACAAACACAGAGACTGGTGCACCCCAGCTCAACCCGATTGACGGACCACTACCTGAAGATAATGAACCAAGTGGGTATGCACAAACAGACTGTGTTCTAGAGGCTATGGCTTTCCTTGAAGAATCCCATCCAGGAATATTTGAAAATTCATGCCTTGAAACAATGGAAGTTGTTCAACAAACAAGGGTAGATAAACTGACTCAAGGTCGCCAGACTTATGATTGGACATTAAACAGAAATCAACCGGCAGCAACTGCATTGGCCAACACCATAGAAGTTTTCAGATCGAATGACCTAACAGCTAACGAGTCAGGAAGGCTAATAGATTTCTTAAAGGATGTGATGGAATCAATGAACAAAGAGGAAATAGAGATAACAACCCACTTTCAAAGAAAAAGGAGAGTAAGAGACAACATGACCAAGAAGATGATCACGCAAAGAACAATAGGGAAGAAAAAACAGAGACTGAATAAGAGAGGCTATCTAATAAGAGCACTGACATTAAATACGATGACCAAAGATGCAGAGAGAGGCAAGTTAAAAAGAAGGGCTATCGCAACACCTGGGATGCAGATTAGAGGTTTCGTATACTTTGTTGAAACTTTAGCTAGGAGCATTTGCGAAAAGCTTGAACAGTCTGGGCTCCCAGTAGGGGGCAATGAAAAGAAGGCCAAATTGGCAAATGTTGTGAGAAAGATGATGACTAATTCACAAGATACAGAGATTTCTTTCACAATCACTGGGGACAACACTAAATGGAATGAGAATCAAAATCCTCGAATGTTCCTGGCGATGATTACATATATCACCAGAAATCAACCCGAGTGGTTTAGAAACATCCTGAGCATGGCACCCATAATGTTCTCAAATAAAATGGCAAGACTAGGGAAAGGGTACATGTTCGAGAGTAAAAGAATGAAGATTCGAACACAAATACCAGCAGAAATGCTAGCAAGCATTGATCTAAAGTATTTCAATGAATCAACAAGGAAGAAAATTGAGAAGATAAGGCCTCTTTTAATGGATGGCACAGCATCACTGAGTCCTGGGATGATGATGGGCATGTTCAACATGCTAAGTACTGTCTTGGGAGTCTCGATACTGAATCTTGGACAAAAGAAATACACCAAGACAACATACTGGTGGGATGGGCTTCAATCATCCGACGATTTTGCTCTCATAGTGAATGCACCAAACCATGAAGGAATACAAGCAGGAGTGGACAGATTCTACAGGACCTGCAAATTAGTGGGAATCAACATGAGCAAAAAGAAGTCCTATATAAATAAGACAGGGACATTTGAATTCACAAGCTTTTTTTATCGCTATGGATTTGTGGCTAATTTTAGCATGGAGCTACCCAGCTTTGGAGTGTCTGGAGTAAATGAATCAGCTGACATGAGTATTGGAGTAACAGTGATAAAGAACAACATGATAAACAATGACCTTGGACCTGCAACGGCTCAGATGGCTCTTCAATTGTTCATCAAAGACTACAGATACACATATAGATGTCATAGGGGAGACACACAAATTCAGACGAGAAGATCATTTGAGTTGAAGAAGCTATGGGATCAAACCCAATCAAAGGTAGGGCTATTAGTATCAGATGGAGGACCAAACTTATACAACATACGGAATCTTCACATTCCTGAAGTCTGCTTAAAATGGGAGCTAATGGATGATGATTATCGGGGAAGACTTTGTAATCCCCTGAATCCCTTTGTGAGTCATAAGGAGATTGATTCTGTAAACAATGCTGTGGTAATGCCAGCCCATGGCCCAGCCAAAAGCATGGAATATGATGCCGTCGCTACTACACATTCCTGGATTCCCAAGAGGAATCGTTCTATTCTCAACACAAGCCAAAGGGGAATTCTTGAGGATGAACAGATGTACCAGAAGTGTTGCAATTTATTCGAGAAATTTTTCCCTAGCAGTTCATATAGGAGGCCGGTTGGAATTTCTAGCATGGTGGAGGCCATGGTGTCTAGGGCCCGGATTGATGCCAGAGTCGACTTTGAGTCTGGACGGATTAAGAAAGAAGAGTTCTCTGAGATCATGAAGATCTGTTCCACCATTGAAGAACTCAGACGGCAAAAATAATGAATTTAACTTGTCCTTCATGAAAAAATG"
print("The final DNA sequence is {} long.".format(len(full_dna_sequence)))

list_of_fragments = []

# The seed ensures, hat everybody gets the same results, when we randomly split the data
random.seed(1)

# The first fragment is also the beginning of the full dna sequence
list_of_fragments.append(full_dna_sequence[0:fragment_length])

# make a number of fragement
for i in range(1,number_of_fragments):

  # start at a random position
  start = random.randint(0, len(full_dna_sequence))

  # end after a defined length
  end = start + fragment_length

  # add the fragement to the list of fragements
  list_of_fragments.append(full_dna_sequence[start:end])
  #print("From {} to {}".format(start, end))

#full_dna_sequence

print("Created a list of {} subquences that are {} long.".format(number_of_fragments,fragment_length))


The final DNA sequence is 2316 long.
Created a list of 500 subquences that are 200 long.


We make the data a litte bit nicer, than it would be in reality:
- First, we add the first and last 200 sequences of the original sequence to the list of fragments:

In [20]:
list_of_fragments.append(full_dna_sequence[0:200])
list_of_fragments.append(full_dna_sequence[-200:])

Next, we ensure, that all fragments have the same length:

In [21]:
for fragment in list_of_fragments:
  if len(fragment)<fragment_length:
    list_of_fragments.remove(fragment)

for fragment in list_of_fragments:
  if len(fragment)<fragment_length:
    list_of_fragments.remove(fragment)

This results in the list of fragments (reads) we are going to work with:

In [22]:
list_of_fragments

['...']

### 2.8.3 Decision on further parameters

Next, we set some further parameters, so that we all work on the same basis.

Frist, we assume, that the want at least 33 nukletides to be identical before we match the data. If this number would become to short, we would falsely connect parts of the data that do not belong together.

If $n$ was 3, both

- ```ABCDEF``` and
- ```ABCXYZ```

could be following to

- ```123ABC```

e.g.,
- ```123ABCDEF```
- ```123ABCXYZ```

This could lead to picking the wrong fragement and lead the algorithm to stop in a dead-end.

As our simple algorithm cannot differentiate what is right. We choose a large $n=33$ as saveguard.

We also make a copy ```list_of_fragments_to_work_with``` of out original data, so we do not mess with it and choose an arbitraty sequence ```longest_fragment``` to start the synthesis. We will use ```longest_fragment``` later to add the matching sequences we found.

In [5]:
# Minimal sequence lenth for valid matches
n = 33

# Copy the list of fragments
list_of_fragments_to_work_with = list_of_fragments[:]

# start with a random fragment
longest_fragment  = list_of_fragments_to_work_with[0]

print("Selected one of the fragments as the inital segment. Now there are {} remaining.".format(len(list_of_fragments_to_work_with)))

Selected one of the fragments as the inital segment. Now there are 454 remaining.


### 2.8.4 🏆 Individual Functions

The challenge is devided into various subproblems. Each subproblem can be solved with a function. If you implement all function, You can pass the assignment even if You would fail to complete the sequence assembly.

The outline of the functions is given below. You can also find a unit test for each function below. 

First, implement end test the following four functions. Run the unit test below, to make sure they work. If You want to get a deeper understanding of unit-tests, look in notebook number 7, however this is optional and not needed to complete the assignment.


In [6]:
def compare_beginning(string, substring, least_overlap):

  """Compares the beginning of the string to see if there is a overlap to the substring. 
  Returns the number of nukleotides that overlap. With the least overlap parameter, 
  You can tweak what the minimal overlap between the strings must have. 
  If the overlap is smaller than the least required overlap, return 0."""
  # ...
  
  return overlap

In [7]:
def compare_end(string, substring,least_overlap):

  """Compares the end of the string to see if there is a overlap to the substring. 
  Returns the number of nukleotides that overlap. If the overlap is smaller than the least required overlap, return 0"""
  # ...
  
  return overlap

In [8]:
def glue_fragments_end(string, substring, size_of_overlap):
  """Returns a String that glues the first part of the substring to the beginning of the string."""
  # ...
  
  return new_string


In [9]:
def glue_fragments_beginning(string, substring, size_of_overlap):
  """Returns a String that glues the first part of the substring to the end of the string."""
  # ...
  
  return new_string

#### 2.8.4.1 🏆 Unit Test 

The following unit test, automatically test Your functions with small examples as an input. 

For instance,  the `test_compare_beginning_find()` is testet by finding the number overlapping letters at the beginning of "QWERTZUIOPASDF" and "AQWERT". The test also passes the parameter "least_overlap=3", so that only overlaps with at least three letters are counted.

If You run the following cell after You defined the function `test_compare_beginning_find()` the following lines should be a result of the cell. 

```
test_compare_beginning_find (__main__.CompareBeginningsTestCase)
Do we find an overlap that is at least 3 at the beginning of the String? ... ok
```

The `ok` indicates, that Your code worked es expected.

In [17]:
import unittest

class CompareBeginningsTestCase(unittest.TestCase):
  """Tests for the challenge"""

  def test_compare_beginning_find(self):
    """Do we find an overlap that is at least 3 at the beginning of the String?"""
    overlap = compare_beginning("QWERTZUIOPASDF", "AQWERT",3)
    self.assertEqual(overlap, 5)
  
  def test_compare_beginning_find_but_useless(self):
    """Do we find if the matching search string we find contributes no additional information?"""
    overlap = compare_beginning("QWERTZUIOPASDF", "QWERTZ",3)
    self.assertEqual(overlap, 0)

  def test_compare_beginning_no_find(self):
    """Do we return 0 if we do not find a match?"""
    overlap = compare_beginning("QWERTZUIOPASDF", "AAAAA",5)
    self.assertEqual(overlap, 0)

class CompareEndingsTestCase(unittest.TestCase):
  """Tests for the challenge"""

  def test_compare_ending_find(self):
    """Do we find an overlap that is at least 3 at the beginning of the String?"""
    overlap = compare_end("QWERTZUIOPASDF", "PASDF12345",3)
    self.assertEqual(overlap, 5)
  
  def test_compare_ending_find_but_useless(self):
    """Do we find if the matching search string we find contributes no additional information?"""
    overlap = compare_end("QWERTZUIOPASDF", "PASDF",4)
    self.assertEqual(overlap, 0)

  def test_compare_ending_no_find(self):
    """Do we return 0 if we do not find a match?"""
    overlap = compare_end("QWERTZUIOPASDF", "PASDF12345",5)
    self.assertEqual(overlap, 0)


class GlueFragmentsTestCase(unittest.TestCase):
  """Tests for the challenge"""

  def test_glue_to_beginning(self):
    """Do we get the right new string in return?"""

    string = "ABCDEGF"
    substring = "0123ABC"
    size_of_overlap = 3

    new_sting = glue_fragments_beginning(string, substring, size_of_overlap)
    new_sting = self.assertEqual(new_sting, "0123ABCDEGF")


  def test_glue_to_end(self):
    """Do we get the right new string in return?"""

    string = "QWERTZUIOPASDF"
    substring = "PASDF12345678"
    size_of_overlap = 5

    new_sting = glue_fragments_end(string, substring, size_of_overlap)
    self.assertEqual(new_sting, "QWERTZUIOPASDF12345678")

unittest.main(argv=[''], verbosity=2, exit=False)

test_compare_beginning_find (__main__.CompareBeginningsTestCase)
Do we find an overlap that is at least 3 at the beginning of the String? ... ok
test_compare_beginning_find_but_useless (__main__.CompareBeginningsTestCase)
Do we find if the matching search string we find contributes no additional information? ... ok
test_compare_beginning_no_find (__main__.CompareBeginningsTestCase)
Do we return 0 if we do not find a match? ... ok
test_compare_ending_find (__main__.CompareEndingsTestCase)
Do we find an overlap that is at least 3 at the beginning of the String? ... ok
test_compare_ending_find_but_useless (__main__.CompareEndingsTestCase)
Do we find if the matching search string we find contributes no additional information? ... ok
test_compare_ending_no_find (__main__.CompareEndingsTestCase)
Do we return 0 if we do not find a match? ... ok
test_glue_to_beginning (__main__.GlueFragmentsTestCase)
Do we get the right new string in return? ... ok
test_glue_to_end (__main__.GlueFragmentsTestC

### 2.8.5 🏆 Putting it all together

Write a final function that uses the former four functions. 

Name the function:
```assemble_sequence(longest_fragment, list_of_fragments_to_work_with)```

This final function takes the `list_of_fragments_to_work_with` and a first `longest_fragment` as arguments and should return a final DNA sequence assembled from the list as a string.
Also calculate the time needed for the assembly and print it out. To do so, You can put the following code in a cell and excecute it after defining the other functions.

```
import time

# get the start time
st = time.time()

result = assemble_sequence(list_of_fragments_to_work_with[0], list_of_fragments_to_work_with[:])
len(result)

print("The final segment is {} nukleotids long.".format(len(result)))


# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st

print('Execution time:', elapsed_time, 'seconds')

```

In [23]:
# add the function here

def assemble_sequence(longest_fragment, list_of_fragments_to_work_with):
  """This final function takes the `list_of_fragments_to_work_with` and a first `longest_fragment` 
  as arguments and should return a final DNA sequence assembled from the list as a string."""
  ...
  final_sequence = longest_fragment # replace this by Your code
  return final_sequence

In [24]:
##  This calls the function

import time

# get the start time
st = time.time()

result = assemble_sequence(list_of_fragments_to_work_with[0], list_of_fragments_to_work_with[:])
len(result)

print("The final segment is {} nukleotids long.".format(len(result)))


# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st

print('Execution time:', elapsed_time, 'seconds')

The final segment is 200 nukleotids long.
Execution time: 0.0009224414825439453 seconds


### Solution

In [13]:
## Solution

def compare_beginning(string, substring, least_overlap):

  """Compares the beginning of the string to see if there is a overlap to the substring. Returns the number of nukleotides that overlap. With the least overlap parameter, You can tweak what the minimal overlap betrween the strings must be."""
  overlap = 0

  # compare the beginning of the string to the substring

  if string[0:len(substring)] == substring:
    its_a_match = True
      # this is useless as we cannot add anything to the beginning
    #print("Have the same length and are indentical.")
  else:
    for length_to_compare in range(len(substring),least_overlap,-1):
        #print(length_to_compare)
        
      last_of_substring = substring[-length_to_compare:]
      first_of_string = string[0:length_to_compare]

      #print("Comparing {} nukleotides: {} - {}".format(length_to_compare,first_of_string,last_of_substring))


      if first_of_string == last_of_substring:
        its_a_match = True
        #print("The first {} nukleotides of the string are identical to the last of the substring!".format(length_to_compare))
        overlap = length_to_compare
        break
  return overlap

def compare_end(string, substring,least_overlap):

  """Compares the end of the string to see if there is a overlap to the substring. Returns the number of nukleotides that overlap."""

  overlap = 0

  # compare the beginning of the string to the substring

  if string[-len(substring):] == substring:
    its_a_match = True
      # this is useless as we cannot add anything to the beginning
    #print("Have the same length and are")
  else:
    for length_to_compare in range(len(substring),least_overlap,-1):
        
      first_of_substring = substring[0:length_to_compare]
      last_of_string = string[-length_to_compare:]


      #print("Comparing {} nukleotides: {} - {}".format(length_to_compare,first_of_substring,last_of_string))

      if first_of_substring == last_of_string:
        its_a_match = True
        #print("The last {} nukleotides of the string are identical to the first of the substring!".format(length_to_compare))
        overlap = length_to_compare
        break
  return(overlap)

def glue_fragments_end(string, substring, size_of_overlap):
  """Returns a String that glues the first part of the substring to the beginning of the string."""
  new_string = string + substring[size_of_overlap:]
  return new_string

def glue_fragments_beginning(string, substring, size_of_overlap):
  """Returns a String that glues the first part of the substring to the end of the string."""
  new_string =  substring[0:len(substring)-size_of_overlap] + string
  return new_string


In [14]:
# go through all remaining fragements and find matching patterns
def assemble_sequence(longest_fragment, list_of_fragments_to_work_with):

  length_of_reconstructed_sequence = len(longest_fragment)

  while length_of_reconstructed_sequence < 2316:
    
    for current_fragement in list_of_fragments_to_work_with:
      print(current_fragement)
    # compare beginnings
      found_overlap = compare_beginning(longest_fragment, current_fragement,n)
      if found_overlap > 0:
        
        #longest_fragment =  current_fragement[0:len(current_fragement)-found_overlap] + longest_fragment
        longest_fragment = glue_fragments_beginning(longest_fragment, current_fragement, found_overlap)
        list_of_fragments_to_work_with.remove(current_fragement)

    # compare endings
      found_overlap = compare_end(longest_fragment, current_fragement,n)
      if found_overlap > 0:
        longest_fragment = glue_fragments_end(longest_fragment, current_fragement, found_overlap)
        #longest_fragment = longest_fragment + current_fragement[-found_overlap:]
        list_of_fragments_to_work_with.remove(current_fragement)

    # if math, find the position and add in to the longest fragement remove the fragment from the list
    length_of_reconstructed_sequence = len(longest_fragment)
    print(len(longest_fragment))

    return longest_fragment


In [15]:
import time

# get the start time
st = time.time()

result = assemble_sequence(list_of_fragments_to_work_with[0], list_of_fragments_to_work_with[:])
len(result)

print("The final segment is {} nukleotids long.".format(len(result)))


# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st

print('Execution time:', elapsed_time, 'seconds')




...
2316
The final segment is 2316 nukleotids long.
Execution time: 0.06664896011352539 seconds


## 2.8.6 Summary

Before You make Your submission, make sure of the following:

- Implement the individual functions under 2.8.4 🏆
- The result cell under the 2.8.4.1 🏆 Unit Test shows that all tests passed (`ok`)
- Write and excute a function to assemble the sequence under 2.8.5 🏆
- After restarting the runtime environment, all the cells in the notebook are run from the top down
- Save the Notebook as an `*.ipynb` and a `*.pdf` file
- Create excecute the last line  in 2.8.6 🏆 to save the final sequence in a `*.txt` file
- Change the name of the three files and upload them to Your dropbox in Sakai

In [16]:
# Stores the final results in a text file

textfile = open("final_result.txt", "w")

textfile.write("> Final Sequence " + "\n")
textfile.write(result)
textfile.close()